In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
pd.options.display.max_columns=1000

# Loading Data

In [102]:
Titanic = sns.load_dataset('titanic')

In [103]:
Titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# Recoding and Dropping

In [104]:
Titanic1 = Titanic.drop('embark_town',axis=1)
Titanic1 = Titanic.drop('class',axis=1)
Titanic1 = Titanic.drop('deck',axis=1)

In [105]:
def sex (series):
    if series == "male":
        return 0
    if series == "female":
        return 1
Titanic1['sexR']=Titanic['sex'].apply(sex)

In [106]:
def alone (series):
    if series == True:
        return 0
    if series == False:
        return 1
Titanic1['aloneR']=Titanic['alone'].apply(alone)

In [107]:
def adult_male (series):
    if series == True:
        return 0
    if series == False:
        return 1
Titanic1['adult_maleR']=Titanic['adult_male'].apply(adult_male)

In [108]:
Titanic.embarked.value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [109]:
def embarked (series):
    if series == "S":
        return 0
    if series == "C":
        return 1
    if series == "Q":
        return 2
Titanic1['embarkedR']=Titanic['embarked'].apply(embarked)

In [110]:
Titanic.who.value_counts()

man      537
woman    271
child     83
Name: who, dtype: int64

In [111]:
def who (series):
    if series == "man":
        return 0
    if series == "woman":
        return 1
    if series == "child":
        return 2
Titanic1['whoR']=Titanic['who'].apply(who)

In [112]:
Titanic.alive.value_counts()

no     549
yes    342
Name: alive, dtype: int64

In [113]:
def alive (series):
    if series == "yes":
        return 0
    if series == "no":
        return 1
Titanic1['aliveR']=Titanic['alive'].apply(alive)

In [114]:
Titanic1.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone,sexR,aloneR,adult_maleR,embarkedR,whoR,aliveR
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False,0,1,0,0.0,0,1
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False,1,1,1,1.0,1,0
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True,1,0,1,0.0,1,0
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False,1,1,1,0.0,1,0
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True,0,0,0,0.0,0,1


In [115]:
Titanic1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  embark_town  889 non-null    object  
 12  alive        891 non-null    object  
 13  alone        891 non-null    bool    
 14  sexR         891 non-null    int64   
 15  aloneR       891 non-null    int64   
 16  adult_maleR  891 non-null    int64   
 17  embarkedR    889 non-null    float64 
 18  whoR         891 non-null    i

In [116]:
Titanic1.dropna(inplace=True)

## Defining X and Y

In [117]:
x = Titanic1[['sexR','aloneR','pclass','age','embarkedR','whoR','adult_maleR','sibsp','fare']]
y = Titanic1['survived']

## Decision Tree

In [118]:
x_train, x_test ,y_train ,y_test = train_test_split(x,y, test_size = 0.4)

In [119]:
decisionTree = DecisionTreeClassifier()
decisionTree.fit(x_train,y_train)

DecisionTreeClassifier()

In [120]:
treePredictions = decisionTree.predict(x_test)
print(confusion_matrix(y_test, treePredictions))

[[133  33]
 [ 33  86]]


In [121]:
print(classification_report(y_test,treePredictions))

              precision    recall  f1-score   support

           0       0.80      0.80      0.80       166
           1       0.72      0.72      0.72       119

    accuracy                           0.77       285
   macro avg       0.76      0.76      0.76       285
weighted avg       0.77      0.77      0.77       285



## Random Forest

In [122]:
forest = RandomForestClassifier()
forest.fit(x_train,y_train)

RandomForestClassifier()

In [123]:
print(accuracy_score(y_test,forest.predict(x_test)))

0.8035087719298246


In [124]:
print(classification_report(y_test,forest.predict(x_test)))

              precision    recall  f1-score   support

           0       0.81      0.86      0.84       166
           1       0.79      0.72      0.75       119

    accuracy                           0.80       285
   macro avg       0.80      0.79      0.80       285
weighted avg       0.80      0.80      0.80       285

